In [1]:
import pandas as pd
import numpy as np

In [2]:
def fir_them(x1, y1, G, S):
    x2  = x1 + S * np.sin(G * np.pi / 200)
    y2 = y1 + S * np.cos(G * np.pi / 200)
    dx = x2 - x1
    dy = y2 - y1
    return (x2, y2, dx, dy)

def sec_them(x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    G1 = np.arctan(abs(dx/dy)) * 200 / np.pi

    if dx > 0 and dy > 0:
        G = G1

    elif dx > 0 and dy < 0:
        G = 200 - G1

    elif dx < 0 and dy < 0:
        G = 200 + G1

    elif dx < 0 and dy > 0:
        G = 400 - G1
    
    elif dx == 0 and dy < 0:
        G = 200
    
    elif dx == 0 and dy > 0:
        G = 0
        
    elif dx < 0 and dy == 0:
        G = 300
        
    elif dx > 0 and dy == 0:
        G = 100

    elif dx == 0 and dy == 0:
        G = np.nan
        
    S = np.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
    return G, S

def third_them(b, G, n, truth = False):
    if truth:
        G2 = G + b + n * 200
    else:
        G2 = G + b + 200
    if G2 > 400:
        k = G2 // 400
        G2 = G2 - k * 400
    return G2

# def rev_G(G):

def dist(x1, y1, x2, y2):
    return(np.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2))

def traverse(x1, y1, x2, y2, x3, y3, x4, y4, s, b, corr = True):
    GT1T2, sT1T2 = sec_them(x1, y1, x2, y2)
    GT3T4, sT3T4 = sec_them(x3, y3, x4, y4)

    dx_pr = x3 - x2
    dy_pr = y3 - y2

    S_sum = np.sum(s)
    b_sum = np.sum(b)

    wb = 5 * np.sqrt(n)
    Ds = 0.02 * np.sqrt(S_sum) + 0.2

    G_truth = third_them(b_sum, GT1T2, n, True)

    Wb = GT3T4 - G_truth
    if Wb < wb and corr:
        print('Angle Check:', np.round(abs((GT3T4 - G_truth) * 100), 2), '<', np.round(wb, 2))
    
    if corr:
        cor_angle = Wb / n
    else:
        cor_angle = 0

    G = np.zeros(len(b))
    G[0] = third_them((b[0]  + cor_angle), GT1T2, n)
    for i in range(1, len(G)):
        G[i] = np.round(third_them((b[i] + cor_angle), G[i - 1], n), 4)

    G = np.insert(G, 0, GT1T2)

    X0 = np.zeros(len(b))
    Y0 = np.zeros(len(b))
    X0[0], Y0[0] = fir_them(x1, y1, G[1], s[0])[2:]

    for i in range(0, len(G) - 1):
        X0[i], Y0[i] = fir_them(X0[i - 1], Y0[i - 1], G[i], s[i - 1])[2:]

    dx_truth = np.sum(X0[1:])
    dy_truth = np.sum(Y0[1:])
    dx = dx_pr - dx_truth
    dy = dy_pr - dy_truth

    ds = np.sqrt(dx ** 2 + dy ** 2)
    if ds < Ds and corr:
        print('Linear Check:', np.round(ds, 2), '<', np.round(Ds, 2))
    
    bwt_x = s/S_sum * dx
    bwt_y = s/S_sum * dy
    bwt_x = np.zeros(len(bwt_x))
    bwt_y = np.zeros(len(bwt_y))

    X = np.zeros(len(b) - 1)
    Y = np.zeros(len(b) - 1)

    X[0] = x2
    Y[0] = y2

    for i in range(1, len(G) - 2):
        X[i] = X0[i] + bwt_x[i - 1] + X[i - 1]
        Y[i] = Y0[i] + bwt_y[i - 1] + Y[i - 1]

    X = np.insert(X, 0, x1)
    Y = np.insert(Y, 0, y1)
    X = np.insert(X, len(X), x3)
    Y = np.insert(Y, len(Y), y3)
    X = np.insert(X, len(X), x4)
    Y = np.insert(Y, len(Y), y4)

    stops = ['T1', 'T2']
    stops[2:] = list(np.arange(1, len(s)+1))
    stops[-1:] = ['T3', 'T4']

    coords = pd.DataFrame(data = [X, Y], index = ['X', 'Y'], columns = stops).T.round(decimals=3)

    X_ch = np.zeros(len(b))
    Y_ch = np.zeros(len(b))

    X_ch[0] = x2
    Y_ch[0] = y2

    for i in range(1, len(G) - 1):
        X_ch[i] = X0[i] + bwt_x[i - 1] + X_ch[i - 1]
        Y_ch[i] = Y0[i] + bwt_y[i - 1] + Y_ch[i - 1]

    X_ch = np.insert(X_ch, 0, x1)
    Y_ch = np.insert(Y_ch, 0, y1)
    X_ch = np.insert(X_ch, len(X_ch), x4)
    Y_ch = np.insert(Y_ch, len(Y_ch), y4)

    coords_check = pd.DataFrame(data = [X_ch, Y_ch], index = ['X', 'Y'], columns = stops).T
    return coords.round(3), coords_check.round(3)

def check_angle_false(x1, y1, x2, y2, x3, y3, x4, y4, s, b):
    trav1 = traverse(x1, y1, x2, y2, x3, y3, x4, y4, s, b, corr = False)[1]
    x4, y4 = x1, y1
    x3, y3 = x2, y2
    x2, y2 = x3, y3
    x1, y1 = x4, y4

    stops = ['T1', 'T2']
    stops[2:] = list(np.arange(1, len(s)+1))
    stops[-1:] = ['T3', 'T4']
    b = [400 - x for x in b]
    b = list(reversed(b))
    s = list(reversed(s))
    stops_rev = list(reversed(stops))
    
    trav2 = traverse(x1, y1, x2, y2, x3, y3, x4, y4, s, b, corr = False)[1]
    trav2.index = stops_rev
    dd = trav1 - trav2
    false = np.sqrt(dd['X'] ** 2 + dd['Y'] ** 2).round(3)
    false = false.reindex(stops)
    false = pd.DataFrame(false, columns = ['Difference (m)'])
    return (false)

In [3]:
x1, y1 = 281.73, 5818.96
x2, y2 = 1248.42, 7143.24
x3, y3 = 1866.37, 7124.23
x4, y4 = 2332.15, 8626.04

s = [112.43, 137.12, 124.19, 119.79, 127.16]

b = [270.135, 187.428, 208.572, 189.114, 205.805, 118.018]

n = len(b)

# x1, y1 = 400661.16, 4540604.717
# x2, y2 = 400726.724, 4540680.44
# x3, y3 = 400726.724, 4540680.440
# x4, y4 = 400661.160, 4540604.717

# s = [31.373, 27.201, 18.392, 25.283, 46.098, 36.615, 31.548, 15.303, 23.384, 29.968]

# b = [220.2089, 269.0663, 181.7531, 212.7550, 296.0945, 312.3851, 129.7549, 270.7580, 211.5708, 282.2047, 13.4706]

# n = len(b)

traverse(x1, y1, x2, y2, x3, y3, x4, y4, s, b)[0]

Angle Check: 6.91 < 12.25
Linear Check: 0.37 < 0.7


,X,Y
T1,281.730,5818.960
T2,1248.420,7143.240
1,1359.391,7125.188
2,1496.420,7130.178
3,1620.014,7118.018
4,1739.484,7126.768
T3,1866.370,7124.230
T4,2332.150,8626.040
